In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, Optional

import cfd_common as comm

NI=comm.NI
NJ=comm.NJ
NIM=NI-1

In [ ]:
@dataclass
class DomainInfo:
    L: float
    NI: int
    NJ: int
    IDIM: int
    JDIM: int
    IJDIM: int
    NIM: int
    NJM: int
    DX0: float
    DY0: float
    
def set_domain(NI, NJ, DX0, DY0):
    global domain

    IDIM = NI + 2
    JDIM = NJ + 2
    IJDIM = max(IDIM, JDIM)
    NIM   = NI - 1
    NJM   = NJ - 1

    domain = DomainInfo(
        NI=NI, NJ=NJ,
        IDIM=IDIM, JDIM=JDIM,
        IJDIM=IJDIM, NIM=NIM, NJM=NJM,
        DX0=DX0, DY0=DY0
    )

@dataclass
class BoundaryValues:
    W: float
    E: float
    S: float
    N: float

@dataclass
class FieldEquation:
    name: str
    # Quantity
    PHI: np.ndarray               
    # Source Term  
    SOR: np.ndarray
    # Coefficients
    AP: np.ndarray
    AW: np.ndarray
    AE: np.ndarray
    AS: np.ndarray
    AN: np.ndarray
    # Relaxation Factor
    relax: float
    # Boundary Cond
    BC: BoundaryValues
    # Neumann Option
    neumann: Optional[Dict[str, Optional[str]]] = None
    def __getitem__(self, idx):
        return self.PHI[idx]     # e.g., phi[i], phi[i,j]

    def __setitem__(self, idx, value):
        self.PHI[idx] = value   # e.g., phi[i,j] = value

In [ ]:
# T = FieldEquation(
#     name = "T",
#     phi = np.zeros((IDIM,JDIM)),
#     SOR = np.zeros((IDIM,JDIM)),
#     AP  = np.zeros((IDIM,JDIM)),
#     AW  = np.zeros((IDIM,JDIM)),
#     AE  = np.zeros((IDIM,JDIM)),
#     AS  = np.zeros((IDIM,JDIM)),
#     AN  = np.zeros((IDIM,JDIM)),
#     bc  = BoundaryValues(W=TW, E=TE, S=TS, N=TN),
#     neumann = {"W": None, "E": None, "S": "neumann", "N": None},
#     relax = 0.8,
# )


In [ ]:
def set_BC(PHI):
    for j in range(0, NJ): PHI[0][j]  =  PHI.BC.W  # West Wall
    for j in range(0, NJ): PHI[NI][j] =  PHI.BC.E  # East Wall
    for i in range(0, NI): PHI[i][0]  =  PHI.BC.S  # South Wall
    for i in range(0, NI): PHI[i][NJ] =  PHI.BC.N  # North Wall

# set_BC(T)
# set_BC(U)
# set_BC(V)
# set_BC(P)

In [ ]:
def set_NEUMANN(PHI, flag):
    if PHI.neumann is None:
        return
    
    if flag == "pre":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": 
                PHI.AP[1][j] -= PHI.AW[1][j]
                PHI.AW[1][j] = 0
            if PHI.neumann.get("E") == "neumann": 
                PHI.AP[NIM][j] -= PHI.AE[NIM][j]
                PHI.AE[NIM][j] = 0
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": 
                PHI.AP[i][1] -= PHI.AS[i][1]
                PHI.AS[i][1] = 0
            if PHI.neumann.get("N") == "neumann": 
                PHI.AP[i][NJM] -= PHI.AN[i][NJM]
                PHI.AN[i][NJM] = 0 
        return

    if flag == "post":
        for j in range(1, NJ):
            if PHI.neumann.get("W") == "neumann": PHI[0][j]  = PHI[1][j]
            if PHI.neumann.get("E") == "neumann": PHI[NI][j] = PHI[NIM][j]
        for i in range(1, NI):
            if PHI.neumann.get("S") == "neumann": PHI[i][0]  = PHI[i][1]
            if PHI.neumann.get("N") == "neumann": PHI[i][NJ] = PHI[i][NJM]
        return

In [ ]:
def minmod(a, b):
    if a * b <= 0:  return 0.0
    if a < 0:       return np.fmax(a, b)
    else:           return np.fmin(a, b)


def get_upwind(i, j, VEL):
    if VEL.name not in ["U", "V"]: AssertionError
    if VEL[i][j] < 0:
        if VEL.name == "U": return (i + 1, j)
        else:               return (i, j + 1)
    return (i,j)


def get_faceVal(PHI, i, j, VEL, scheme=None):  # MUSCL ALGORITHM
    (u_i, u_j) = get_upwind(i, j, VEL)
    if VEL.name == "U":
        r_diff = PHI[u_i + 1][u_j]  - PHI[u_i][u_j]
        l_diff = PHI[u_i][u_j]      - PHI[u_i - 1][u_j]
    if VEL.name == "V":
        r_diff = PHI[u_i][u_j + 1] - PHI[u_i][u_j]
        l_diff = PHI[u_i][u_j]     - PHI[u_i][u_j - 1]
    diff = minmod(r_diff, l_diff)
    sign = (-1) ** ((u_i - i) + (u_j - j))

    PHI_right = PHI[u_i][u_j] + sign * 0.5 * diff

    return PHI_right

def get_RES(PHI):
    global U, V
    global RHS, RES, CORR

    RESMAX = 0
    FFMAX = 0
    FCMAX = 0
    
    # b: Flux Coeff
    if PHI.name=="T":           b=RHO * CP
    elif PHI.name in ["U","v"]: b=RHO
    
    for i in range(1, NI):
        for j in range(1, NJ):
            
            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            # 
            elif PHI.name in ["U","V"]:
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i-1][j])
                vn=0.5*(V[i][j]+V[i+1][j])
            
            # First Order Upwind(Implicit)
            flux_W = b * RU[i]       * DYP[j] * uw
            flux_E = b * RU[i + 1]   * DYP[j] * ue
            flux_S = b * RP[i]       * DXP[i] * vs
            flux_N = b * RP[i]       * DXP[i] * vn

            neighbors = (
                  PHI.AW[i][j] * PHI[i - 1][j]
                + PHI.AE[i][j] * PHI[i + 1][j]
                + PHI.AS[i][j] * PHI[i][j - 1]
                + PHI.AN[i][j] * PHI[i][j + 1]
            )
            FOU = PHI.AP[i][j] * PHI[i][j] - neighbors
        

            if PHI.name=="T":
                x_advect_1 = flux_E * get_faceVal(i, j, "x") - flux_W * get_faceVal(i - 1, j, "x")
                y_advect_1 = flux_N * get_faceVal(i, j, "y") - flux_S * get_faceVal(i, j - 1, "y")

                x_advect_2 = (np.fmax(0, flux_E) * PHI[i][j]   + np.fmax(0, -flux_E) * PHI[i+1][j]) \
                           - (np.fmax(0, flux_W) * PHI[i-1][j] + np.fmax(0, -flux_W) * PHI[i][j])

                y_advect_2 = (np.fmax(0, flux_N) * PHI[i][j]   + np.fmax(0, -flux_N) * PHI[i][j+1]) \
                           - (np.fmax(0, flux_S) * PHI[i][j-1] + np.fmax(0, -flux_S) * PHI[i][j])

                x_advect = x_advect_1 - x_advect_2
                y_advect = y_advect_1 - y_advect_2
                advect = x_advect + y_advect
            elif PHI.name in ["U","V"]:
                

            SOU = FOU + advect

            # residual & correction
            RES[i][j] = RHS[i][j] - SOU
            RESMAX = max(RESMAX, np.fabs(RES[i][j] / PHI.AP[i][j]))
            FFMAX = max(FFMAX, np.fabs(PHI[i][j]))
            FCMAX = max(FCMAX, np.fabs(CORR[i][j]))

    FFMAX = max(1.0e-30, FFMAX)
    RESMAX = RESMAX / FFMAX
    FCMAX = FCMAX / FFMAX

    return (RES, RESMAX, FFMAX, FCMAX)
   

### TODO List

1. Upwind criteria for U,V --> flux-based selection
2. Staggered Grid description --> U,V control volume
3. Non-uniform Grid
4. Theory 1, Theory 2

In [ ]:
def SOLVE(PHI):
    global U, V, P
    global RHS
    
    if PHI.name not in ["U","V","P","T"]: AssertionError
    RHS=np.zeros((IDIM,JDIM))

    # TDMA coeffs
    for i in range(1, NI):
        for j in range(1, NJ):
            
            # a: Advection Coeff
            if PHI.name=="T":           a=K0
            elif PHI.name in ["U","V"]: a=MU0
            
            _AW = a * RU[i] * DYP[j] / DXU[i]
            _AE = a * RU[i + 1] * DYP[j] / DXU[i + 1]
            _AS = a * RP[i] * DXP[i] / DYV[j]
            _AN = a * RP[i] * DXP[i] / DYV[j + 1]

            if PHI.name=="T":  
                uw=U[i][j]
                ue=U[i+1][j]
                vs=V[i][j]
                vn=V[i+1][j]
            
            # U,V face-velocity --> upwind scheme
            elif PHI.name in ["U","V"]:
                uw=0.5*(U[i][j]+U[i-1][j])
                ue=0.5*(U[i][j]+U[i+1][j])
                vs=0.5*(V[i][j]+V[i-1][j])
                vn=0.5*(V[i][j]+V[i+1][j])
                
            # b: Flux Coeff
            if PHI.name=="T":           b=RHO * CP
            elif PHI.name in ["U","v"]: b=RHO
            
            # First Order Upwind(Implicit)
            flux_W = b * RU[i]       * DYP[j] * uw
            flux_E = b * RU[i + 1]   * DYP[j] * ue
            flux_S = b * RP[i]       * DXP[i] * vs
            flux_N = b * RP[i]       * DXP[i] * vn

            PHI.AW[i][j] = _AW + np.fmax(0, +flux_W)
            PHI.AE[i][j] = _AE + np.fmax(0, -flux_E)
            PHI.AS[i][j] = _AS + np.fmax(0, +flux_S)
            PHI.AN[i][j] = _AN + np.fmax(0, -flux_N)
            PHI.AP[i][j] = PHI.AW[i][j] + PHI.AE[i][j] + PHI.AS[i][j] + PHI.AN[i][j]
            
            if PHI.name == "T": 
                RHS[i][j] = QDOT * RP[i] * DXP[i] * DYP[j]
            elif PHI.name == "U":
                dV = RP[i][j] * DYV[i][j] * DXP[i][j]
                diff_l=(P[i][j]-P[i-1][j])/DXU[i]
                diff_r=(P[i+1][j]-P[i][j])/DXU[i+1]
                RHS[i][j] -= 0.5*(diff_l+diff_r)*dV
            elif PHI.name == "V":
                dV = RP[i][j] * DXU[i][j] * DYP[i][j]
                diff_l=(P[i][j]-P[i-1][j])/DYV[i]
                diff_r=(P[i+1][j]-P[i][j])/DYV[i+1]
                RHS[i][j] -= 0.5*(diff_l+diff_r)*dV
                
            
    # boundary conds
    set_NEUMANN(PHI, "pre")

    # earn T_hat (T_numerical)
    for iter in range(ITERMAX):
        (RES, RESMAX, FFMAX, FCMAX) = get_RES()
        if monitor_RES(iter, RESMAX, FCMAX) == 0:
            break

        CORR = LLTDMA(1, NIM, 1, NJM, AW, AE, AS, AN, AP, SOR=RES)
        for i in range(1, NI):
            for j in range(1, NJ):
                PHI[i][j] += PHI.relax * CORR[i][j]

        set_NEUMANN(PHI, "post")

    return PHI
